<a href="https://colab.research.google.com/github/maanqii/coding-three/blob/main/ml/CartoonGAN_TFLite_Fixed_Shaped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates the TFLite model conversion process for the **White-box Cartoonization** model as proposed in the paper [Learning to Cartoonize Using White-box Cartoon Representations](https://github.com/SystemErrorWang/White-box-Cartoonization/blob/master/paper/06791.pdf). Below you can find an example output that this model is capable of producing:

![](https://i.ibb.co/DkzL9J7/Screen-Shot-2020-07-21-at-1-42-25-PM.png)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;[GitHub repository of the paper](https://github.com/SystemErrorWang/White-box-Cartoonization)


Code for the paper is available here in [this repository](https://github.com/SystemErrorWang/White-box-Cartoonization/).


**Acknowledgements**: Khanh LeViet

## Setup and imports

As the main model is implemented in TensorFlow 1 it's important for us replicate a TensorFlow 1.x runtime to be able to load the models weights, perform inference, and export a `SavedModel`.

In [5]:
!git clone https://github.com/SystemErrorWang/White-box-Cartoonization

fatal: destination path 'White-box-Cartoonization' already exists and is not an empty directory.


In [6]:
import sys
sys.path.append('./White-box-Cartoonization/test_code')

import matplotlib.pyplot as plt
import cartoonize
import os

from PIL import Image

ModuleNotFoundError: ignored

## Gather sample data and run inference to make sure the model works

In [ ]:
!mkdir ./source-frames
!wget -O image.jpg https://raw.githubusercontent.com/sayakpaul/portfolio/master/images/image_sayak.jpg
!mv image.jpg ./source-frames/

In [ ]:
model_path = './White-box-Cartoonization/test_code/saved_models'
load_folder = './source-frames'
save_folder = './cartoonized_images'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

cartoonize.cartoonize(load_folder, save_folder, model_path)

source_image = plt.imread('./source-frames/image.jpg')
cartoonized_image = plt.imread('./cartoonized_images/image.jpg')

plt.subplot(1, 2, 1)
plt.imshow(source_image)
plt.title('Source image')
plt.subplot(1, 2, 2)
plt.imshow(cartoonized_image)
plt.title('Cartoonized image')
plt.show()

## Create a `SavedModel` from the pre-trained checkpoints

In [ ]:
# Directory where SavedModel is to be saved
!mkdir saved_model_dir

In [ ]:
# Comes with the GitHub repo
# These will be needed to instantiate the model
import network
import guided_filter

In [ ]:
# We will be using TensorFlow session
import tensorflow as tf
print(tf.__version__)

In [ ]:
tf.reset_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
  # Create placeholder for the input
  input_photo = tf.placeholder(tf.float32, [1, None, None, 3], name='input_photo')

  # Run the input placeholder through the generator, and then apply a
  # filter to process the generator output
  network_out = network.unet_generator(input_photo)
  final_out = guided_filter.guided_filter(input_photo, network_out, r=1, eps=5e-3)
  final_out = tf.identity(final_out, name='final_output') # Create an identical filtering layer

  # The above process is basically needed to construct the computation graph for the
  # current session

  # Get the generator variables and restore the pre-trained checkpoints in the
  # current session
  all_vars = tf.trainable_variables()
  gene_vars = [var for var in all_vars if 'generator' in var.name]
  saver = tf.train.Saver(var_list=gene_vars)
  sess.run(tf.global_variables_initializer())
  saver.restore(sess, tf.train.latest_checkpoint(model_path))

  # Export to SavedModel
  tf.saved_model.simple_save(
      sess,
      '/content/saved_model_dir',
      inputs={input_photo.name: input_photo},
      outputs={final_out.name: final_out})

### Inspect the files sizes

In [ ]:
!ls -lh saved_model_dir
!ls -lh saved_model_dir/variables

In [ ]:
!ls -lh /content/White-box-Cartoonization/test_code/saved_models

**A runtime restart is required here.**

# TFLite model conversion

## Imports

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

## Dynamic-range quantization

In [ ]:
model = tf.saved_model.load('saved_model_dir')
concrete_func = model.signatures[
    tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 512, 512, 3]) # Just comment this line if you want to export a model with dynamic shape support

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

open('whitebox_cartoon_gan_dr.tflite', 'wb').write(tflite_model)

In [ ]:
!ls -lh whitebox_cartoon_gan_dr.tflite

## float16 quantization

We further reduce the size of the inputs for float16 model so that it runs reasonably faster on an Android device.

In [ ]:
model = tf.saved_model.load('saved_model_dir')
concrete_func = model.signatures[
    tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 224, 224, 3])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

open('whitebox_cartoon_gan_fp16.tflite', 'wb').write(tflite_model)

In [ ]:
!ls -lh whitebox_cartoon_gan_fp16.tflite

## int8 quantization

In [ ]:
# Gather some more data
!wget -O image2.jpg https://storage.googleapis.com/khanhlvg-public.appspot.com/arbitrary-style-transfer/belfry-2611573_1280.jpg
!wget -O image3.jpg https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg
!wget -O image4.jpg https://pbs.twimg.com/profile_images/1260205070555086855/erU_iILT_400x400.jpg
!wget -O image5.jpg https://pbs.twimg.com/profile_images/1235595938921459713/h26CpAPb_400x400.jpg

In [ ]:
# Put these images in a central directory
!cp -r *.jpg source-frames/

In [ ]:
IMG_SIZE = 512
images_list = os.listdir('/content/source-frames')

# int8 quantization requires a representative dataset generator
def representative_dataset_gen():
    for image_path in images_list:
        image = cv2.imread(os.path.join('/content/source-frames', image_path))
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
        image = image.astype(np.float32)/127.5 - 1
        image = np.expand_dims(image, axis=0)
        yield [image]

model = tf.saved_model.load('saved_model_dir')
concrete_func = model.signatures[
    tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, IMG_SIZE, IMG_SIZE, 3])

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()

open('whitebox_cartoon_gan_int8.tflite', 'wb').write(tflite_model)

In [ ]:
!ls -lh whitebox_cartoon_gan_int8.tflite

## Sample inference

In [ ]:
# Reference: https://www.tensorflow.org/lite/models/style_transfer/overview
def load_img(path_to_img):
  img = cv2.imread(path_to_img)
  img = img.astype(np.float32) / 127.5 - 1
  img = np.expand_dims(img, 0)
  img = tf.convert_to_tensor(img)
  return img

# Function to pre-process by resizing an central cropping it.
def preprocess_image(image, target_dim=224):
  # Resize the image so that the shorter dimension becomes the target dim.
  shape = tf.cast(tf.shape(image)[1:-1], tf.float32)
  short_dim = min(shape)
  scale = target_dim / short_dim
  new_shape = tf.cast(shape * scale, tf.int32)
  image = tf.image.resize(image, new_shape)

  # Central crop the image.
  image = tf.image.resize_with_crop_or_pad(image, target_dim, target_dim)

  return image

In [ ]:
model_type = "int8" #@param ["dr", "int8", "float16"]
source_image = load_img('/content/source-frames/image.jpg')
if model_type == "float16":
    preprocessed_source_image = preprocess_image(source_image, target_dim=224)
else:
    preprocessed_source_image = preprocess_image(source_image, target_dim=512)
preprocessed_source_image.shape

In [ ]:
#@title Run inference
model_dict = {
    "dr": "/content/whitebox_cartoon_gan_dr.tflite",
    "int8": "/content/whitebox_cartoon_gan_int8.tflite",
    "float16": "/content/whitebox_cartoon_gan_fp16.tflite"
}

interpreter = tf.lite.Interpreter(model_path=model_dict[model_type])
input_details = interpreter.get_input_details()

# Comment out the following when dealing with models with dynamic shaped inputs
# if model_type == 'int8':
#     interpreter.resize_tensor_input(156, [1, preprocessed_source_image.shape[1], preprocessed_source_image.shape[2], 3])
# else:
#     interpreter.resize_tensor_input(0, [1, preprocessed_source_image.shape[1], preprocessed_source_image.shape[2], 3])

interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], preprocessed_source_image)
interpreter.invoke()

raw_prediction = interpreter.tensor(
    interpreter.get_output_details()[0]['index'])()

## Process the output

In [ ]:
output = (np.squeeze(raw_prediction)+1.0)*127.5
output = np.clip(output, 0, 255).astype(np.uint8)
output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)

In [ ]:
plt.subplot(1, 2, 1)
plt.imshow(plt.imread('/content/source-frames/image.jpg'))
plt.title('Source image')
plt.subplot(1, 2, 2)
plt.imshow(output)
plt.title('Cartoonized image')
plt.show()